In [9]:
%%capture

!pip install b2bTools

In [14]:
# !pip install --upgrade mdtraj
# !pip install --upgrade b2btools

In [1]:
# !pip install gemmi ipywidgets requests pandas
import re, json, requests, pandas as pd
from gemmi import cif
from IPython.display import HTML, display
import ipywidgets as W

# ---------- UniProt helpers ----------
def fetch_uniprot_entry(acc):
    r = requests.get(f"https://rest.uniprot.org/uniprotkb/{acc}.json",
                     headers={"Accept":"application/json","User-Agent":"scop3p-nb/1.0"}, timeout=30)
    r.raise_for_status()
    return r.json()

def pdb_chains_from_uniprot(acc):
    """Return dict {PDB_ID: {'auth_chains': set(), 'label_chains': set()}} from UniProt xrefs + fallback from mmCIF."""
    j = fetch_uniprot_entry(acc)
    xrefs = j.get("uniProtKBCrossReferences", [])
    pdbs = [x for x in xrefs if x.get("database") == "PDB"]
    out = {}
    # Parse 'chains' field like "A=1-300; B=..." to get auth chains quickly
    for x in pdbs:
        pid = x.get("id")
        props = {p["key"]: p["value"] for p in x.get("properties", [])}
        chains_str = props.get("Chains") or props.get("chains") or ""
        auth = set(re.findall(r"([A-Za-z0-9]+)=", chains_str))
        out.setdefault(pid, {"auth_chains": set(), "label_chains": set()})
        out[pid]["auth_chains"] |= auth
    return out

# ---------- Scop3P ----------
def fetch_scop3p_mods(accession):
    url = f"https://iomics.ugent.be/scop3p/api/modifications?accession={accession}"
    r = requests.get(url, headers={"Accept":"application/json"}, timeout=30)
    r.raise_for_status()
    obj = r.json()
    mods = obj.get("modifications", [])
    # phospho (you can adapt filter)
    phospho = sorted({m["position"] for m in mods if "phospho" in (m.get("name","").lower())})
    return mods, phospho

# ---------- PDBe updated mmCIF ----------
def get_updated_mmcif_text(pdb_id):
    pdb = pdb_id.lower()
    for u in (f"https://www.ebi.ac.uk/pdbe/entry-files/{pdb}_updated.cif",
              f"https://www.ebi.ac.uk/pdbe/entry-files/download/{pdb}_updated.cif",
              f"https://www.ebi.ac.uk/pdbe/static/entry/{pdb}_updated.cif"):
        r = requests.get(u, timeout=60)
        if r.ok and r.text.strip():
            return r.text
    raise RuntimeError(f"Could not retrieve updated CIF for {pdb_id}")

def read_mmcif_category(block, cat):
    """Return a pandas DataFrame for a given category name using gemmi's iterator interface."""
    tbl = block.get_mmcif_category(cat)
    return pd.DataFrame(tbl) if tbl is not None else pd.DataFrame()

def parse_updated_mmcif_for_chain(pdb_id, uniprot_acc, chain_choice):
    """
    Parse updated mmCIF and return:
      up2pdb: { unp_pos: [ { 'pdb', 'chain_label','label_seq_id','chain_auth','auth_seq_id','icode' } ... ] }
      pfam_segments: [{uniprot_acc,pfam_id,unp_start,unp_end}], cath_segments: similar
    chain_choice: can be author chain (e.g., 'A') or label chain (e.g., 'A'); we match against both.
    """
    text = get_updated_mmcif_text(pdb_id)
    doc = cif.read_string(text)
    block = doc.sole_block()

    # Residue numbering table (more compact than atom_site): _pdbx_poly_seq_scheme
    poly = read_mmcif_category(block, "_pdbx_poly_seq_scheme")
    # Columns we need may be named slightly differently across entries
    # Normalize expected columns
    colmap = {
        "asym_id": "asym_id",
        "seq_id": "seq_id",
        "auth_seq_num": "auth_seq_num",
        "pdb_ins_code": "pdb_ins_code",
        "auth_asym_id": "auth_asym_id",
    }
    for need in list(colmap):
        if need not in poly.columns:
            # try common variants
            if need == "auth_seq_num" and "auth_seq_id" in poly.columns:
                poly["auth_seq_num"] = poly["auth_seq_id"]
            elif need == "pdb_ins_code" and "pdbx_PDB_ins_code" in poly.columns:
                poly["pdb_ins_code"] = poly["pdbx_PDB_ins_code"]
            elif need == "auth_asym_id" and "auth_asym_id" not in poly.columns:
                # often missing here; we can fill later from atom_site if needed
                poly["auth_asym_id"] = ""
    poly = poly[["asym_id","seq_id","auth_seq_num","pdb_ins_code","auth_asym_id"]].copy()

    # If auth_asym_id absent, derive from _atom_site (map label->auth chain once)
    if (poly["auth_asym_id"] == "").all():
        atom = read_mmcif_category(block, "_atom_site")
        sub = atom[["label_asym_id","auth_asym_id"]].dropna().drop_duplicates()
        m = dict(sub.values)
        poly["auth_asym_id"] = poly["asym_id"].map(m).fillna("")

    # SIFTS per-residue xref table
    sifts = read_mmcif_category(block, "_pdbx_sifts_xref_db")
    if sifts.empty:
        # as you noted, you can also read from atom_site's per-atom SIFTS if needed,
        # but in most updated files _pdbx_sifts_xref_db exists. We'll keep this simple now.
        return {"up2pdb": {}, "pfam_segments": [], "cath_segments": [], "reason": "no_pdbx_sifts_xref_db"}

    # We only need UNP rows for residue mapping; PFAM/CATH for domain ranges
    sifts = sifts.rename(columns=str)
    # ensure needed cols exist
    needed = ["asym_id","seq_id","unp_acc","unp_num","xref_db_name","xref_db_acc","mon_id_one_letter_code","unp_res"]
    for c in needed:
        if c not in sifts.columns:
            sifts[c] = ""

    # Filter to our UniProt
    unp = sifts[(sifts["unp_acc"] == uniprot_acc) & (sifts["seq_id"] != "") & (sifts["unp_num"] != "")]
    if unp.empty:
        return {"up2pdb": {}, "pfam_segments": [], "cath_segments": [], "reason": "target_uniprot_not_found_in_sifts"}

    # Convert numeric fields
    unp["seq_id"] = pd.to_numeric(unp["seq_id"], errors="coerce").astype("Int64")
    unp["unp_num"] = pd.to_numeric(unp["unp_num"], errors="coerce").astype("Int64")
    unp = unp.dropna(subset=["seq_id","unp_num"]).astype({"seq_id": int, "unp_num": int})
    
    # --- make types compatible for merge ---
    unp["seq_id"]  = unp["seq_id"].astype(str)
    poly["seq_id"] = poly["seq_id"].astype(str)

    # Join: add label+author numbering for this (asym_id, seq_id)
    join = unp.merge(poly, on=["asym_id","seq_id"], how="left")

    # Fallback fill for author numbering from _atom_site if missing
    atom = read_mmcif_category(block, "_atom_site")
    # Deduplicate to residue-level map using label ids
    atom_map = (atom[["label_asym_id","label_seq_id","auth_asym_id","auth_seq_id","pdbx_PDB_ins_code"]]
                .dropna(subset=["label_asym_id","label_seq_id"])
                .drop_duplicates(subset=["label_asym_id","label_seq_id"]))
    atom_map["label_seq_id"] = atom_map["label_seq_id"].astype(str)

    # Merge fallback author info (suffix _atm)
    join = join.merge(
        atom_map,
        left_on=["asym_id","seq_id"],
        right_on=["label_asym_id","label_seq_id"],
        how="left",
        suffixes=("", "_atm")
    )

    # Choose author fields: prefer poly, else atom_site fallback
    def first_nonempty(a, b):
        return a if (isinstance(a, str) and a) or (pd.notna(a) and a!='') else b

    join["auth_asym_id_final"] = [
        first_nonempty(a, b) for a, b in zip(join.get("auth_asym_id", ""), join.get("auth_asym_id_atm", ""))
    ]
    # auth_seq_num may be missing; try atom_site.auth_seq_id
    join["auth_seq_num_final"] = pd.to_numeric(join.get("auth_seq_num"), errors="coerce")
    auth_seq_fallback = pd.to_numeric(join.get("auth_seq_id_atm"), errors="coerce")
    join.loc[join["auth_seq_num_final"].isna(), "auth_seq_num_final"] = auth_seq_fallback

    # insertion code: prefer poly, else atom_site
    icode_poly = join.get("pdb_ins_code")
    icode_atom = join.get("pdbx_PDB_ins_code")
    join["icode_final"] = [
        (x if isinstance(x, str) and x.strip() not in (".","?","") else (y if isinstance(y, str) and y.strip() not in (".","?","") else None))
        for x, y in zip(icode_poly, icode_atom)
    ]

    # restrict to the selected chain (match label or author)
    mask_chain = (join["asym_id"].astype(str) == str(chain_choice)) | (join["auth_asym_id_final"].astype(str) == str(chain_choice))
    join = join[mask_chain].copy()
    if join.empty:
        return {"up2pdb": {}, "pfam_segments": [], "cath_segments": [], "reason": f"no_rows_for_chain_{chain_choice}"}

    # --- build mapping dict (label IDs always present; author optional) ---
    up2pdb = {}
    pdb_l = pdb_id.lower()

    # de-duplicate per residue (one row per unp_num, chain_label, label_seq_id)
    join["label_seq_id_int"] = pd.to_numeric(join["seq_id"], errors="coerce").astype("Int64")
    dedup_keys = ["asym_id","label_seq_id_int","unp_num"]
    join = join.dropna(subset=["label_seq_id_int","unp_num"]).drop_duplicates(subset=dedup_keys)

    for r in join.itertuples(index=False):
        up_pos = int(r.unp_num)
        chain_label = str(r.asym_id)
        label_seq_id = int(r.label_seq_id_int)

        # optional author fields
        chain_auth = str(r.auth_asym_id_final) if pd.notna(r.auth_asym_id_final) and str(r.auth_asym_id_final) else None
        try:
            auth_seq = int(r.auth_seq_num_final) if pd.notna(r.auth_seq_num_final) else None
        except Exception:
            auth_seq = None
        icode = r.icode_final if isinstance(r.icode_final, str) and r.icode_final else None

        entry = {
            "pdb": pdb_l,
            "chain_label": chain_label,
            "label_seq_id": label_seq_id,
        }
        if chain_auth: entry["chain_auth"] = chain_auth
        if auth_seq is not None: entry["auth_seq_id"] = auth_seq
        if icode: entry["icode"] = icode

        up2pdb.setdefault(up_pos, []).append(entry)

    # Domain ranges from SIFTS rows (UniProt coords)
    def _collapse_domain(df, dbname, id_field):
        sub = df[df["xref_db_name"].str.upper().eq(dbname)][["xref_db_acc","unp_num"]].copy()
        sub = sub.dropna()
        if sub.empty: return []
        sub["unp_num"] = sub["unp_num"].astype(int)
        segs = []
        for dom, grp in sub.sort_values(["xref_db_acc","unp_num"]).groupby("xref_db_acc"):
            pos = grp["unp_num"].tolist()
            s = prev = pos[0]
            for p in pos[1:]:
                if p != prev+1:
                    segs.append({"uniprot_acc": uniprot_acc, id_field: dom, "unp_start": s, "unp_end": prev})
                    s = p
                prev = p
            segs.append({"uniprot_acc": uniprot_acc, id_field: dom, "unp_start": s, "unp_end": prev})
        return segs

    pfam_segments = _collapse_domain(sifts, "PFAM", "pfam_id")
    cath_segments = _collapse_domain(sifts, "CATH", "cath_id")

    return {"up2pdb": up2pdb, "pfam_segments": pfam_segments, "cath_segments": cath_segments, "reason": "ok"}

# ---------- AlphaFold check ----------
def alphafold_id_v4(acc):
    af = f"AF-{acc}-F1-model_v4"
    try:
        ok = requests.head(f"https://alphafold.ebi.ac.uk/files/{af}.cif", timeout=15).ok
    except Exception:
        ok = False
    return af if ok else None


In [32]:
# print (PACK)

In [6]:
# --- Robust PDB/CHAIN options from updated mmCIFs ---
import requests, re, pandas as pd
from gemmi import cif
import ipywidgets as W
from IPython.display import display

UNI = UNI if 'UNI' in globals() else "O00571"  # keep your existing UNI if set

def uniprot_pdb_ids(acc):
    url = f"https://rest.uniprot.org/uniprotkb/{acc}.json"
    j = requests.get(url, headers={"Accept":"application/json","User-Agent":"scop3p-nb/1.0"}, timeout=30).json()
    xrefs = j.get("uniProtKBCrossReferences", [])
    return sorted({x["id"] for x in xrefs if x.get("database") == "PDB"})

def get_updated_mmcif_text(pid):
    pid = pid.lower()
    for u in (f"https://www.ebi.ac.uk/pdbe/entry-files/{pid}_updated.cif",
              f"https://www.ebi.ac.uk/pdbe/entry-files/download/{pid}_updated.cif",
              f"https://www.ebi.ac.uk/pdbe/static/entry/{pid}_updated.cif"):
        r = requests.get(u, timeout=30)
        if r.ok and r.text.strip():
            return r.text
    return ""

def read_cat(block, cat):
    tbl = block.get_mmcif_category(cat)
    return pd.DataFrame(tbl) if tbl is not None else pd.DataFrame()

def mmcif_chains(pid):
    """Return sorted unique label chains (preferred) and author chains from updated mmCIF."""
    txt = get_updated_mmcif_text(pid)
    if not txt:
        return [], []
    block = cif.read_string(txt).sole_block()
    poly = read_cat(block, "_pdbx_poly_seq_scheme")
    lab = sorted({str(x) for x in poly.get("asym_id", []) if str(x).strip() not in ("", ".", "?")})
    auth = sorted({str(x) for x in (poly.get("auth_asym_id", []) or []) if str(x).strip() not in ("", ".", "?")})
    # fallback via _atom_site if needed
    if not lab or not auth:
        atom = read_cat(block, "_atom_site")
        if not lab and "label_asym_id" in atom:
            lab = sorted({str(x) for x in atom["label_asym_id"] if str(x).strip() not in ("", ".", "?")})
        if not auth and "auth_asym_id" in atom:
            auth = sorted({str(x) for x in atom["auth_asym_id"] if str(x).strip() not in ("", ".", "?")})
    return lab, auth

def alphafold_id_v4(acc):
    af = f"AF-{acc}-F1-model_v4"
    try:
        return af if requests.head(f"https://alphafold.ebi.ac.uk/files/{af}.cif", timeout=10).ok else None
    except Exception:
        return None

# Build options list
pdb_ids = uniprot_pdb_ids(UNI)
options = []
for pid in pdb_ids:
    label_chains, auth_chains = mmcif_chains(pid)
    chains = label_chains or auth_chains  # prefer label chains for Mol*
    for ch in chains:
        options.append(f"{pid}/{ch}")

af_id = alphafold_id_v4(UNI)
if af_id:
    options = [f"AF:{af_id}"] + options

# Show a quick debug count
print(f"UniProt {UNI}: {len(pdb_ids)} PDB IDs; dropdown options: {len(options)}")

# If still empty, print first few PDB IDs to investigate
if not options:
    print("No chain options could be built. First few PDB IDs from UniProt:", pdb_ids[:10])

# Make the widget
sel = W.Dropdown(options=options, description="Structure", layout=W.Layout(width="460px"))
btn = W.Button(description="Load selection", layout=W.Layout(width="200px"))
out = W.Output()

display(sel, btn, out)


def build_PACK_for_selection(choice):
    """Return PACK for the selected PDB/chain or AF model."""
    if choice.startswith("AF:"):
        PACK = {
            "uniprot": UNI,
            "structures": {"alphafold": choice.split("AF:",1)[1], "pdbs": list(pdbs.keys())},
            "tracks": {"conservation": [], "disorder": [], "plddt": []},  # fill with your arrays if available
            "scop3p": {"phospho_positions": phospho_positions},
            "pfam": [], "cath": [],
            "mapping": {}  # AF uses 1:1 mapping by default in the UI code
        }
        return PACK

    pdb_id, chain = choice.split("/")
    # Parse updated mmCIF for this chain
    mapping = parse_updated_mmcif_for_chain(pdb_id, UNI, chain)
    # Build PACK
    PACK = {
        "uniprot": UNI,
        "structures": {"alphafold": af_id, "pdbs": list(pdbs.keys())},
        "tracks": {"conservation": [], "disorder": [], "plddt": []},  # plug your biophysical arrays here
        "scop3p": {"phospho_positions": phospho_positions},
        "pfam": mapping["pfam_segments"],
        "cath": mapping["cath_segments"],
        "mapping": mapping["up2pdb"]
    }
    return PACK

from IPython.display import HTML, display
import html, json, os
from IPython.display import HTML, display
import html, json

def render_PACK(PACK):
    page = r"""<!doctype html>
<html>
<head>
<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<title>ProtVista + Mol*</title>
<style>
  body { margin:0; font-family: ui-sans-serif, system-ui; }
</style>
</head>
<body>
<div style="display:grid;grid-template-columns:460px 1fr;gap:14px;height:100vh;">
  <div style="border-right:1px solid #ddd;padding:10px;overflow:auto">
    <div style="display:flex;gap:10px;align-items:center;margin-bottom:8px">
      <label>UniProt accession:</label>
      <input id="acc" value="__UNIPROT__" style="padding:6px 8px;border:1px solid #ccc;border-radius:8px;width:140px">
    </div>

    <div style="display:flex;gap:10px;align-items:center;margin-bottom:10px">
      <label>Structure (select with Python widget):</label>
      <select id="structSel" style="padding:6px 8px;border:1px solid #ccc;border-radius:8px;min-width:260px"></select>
    </div>

    <!-- ProtVista (UMD builds) -->
    <script src="https://unpkg.com/protvista-uniprot@latest/dist/protvista-uniprot.min.js"></script>
    <script src="https://unpkg.com/protvista-tracks@latest/dist/protvista-tracks.min.js"></script>

    <protvista-uniprot id="pv-u" accession="__UNIPROT__"></protvista-uniprot>

    <h4 style="margin:14px 0 6px">Custom tracks</h4>
    <protvista-track id="track-cons"  length="0" displaystart="1" displayend="0" shape="rectangle" label="Conservation"        layout="compact"></protvista-track>
    <protvista-track id="track-diso"  length="0" displaystart="1" displayend="0" shape="rectangle" label="Disorder (DisoMine)" layout="compact"></protvista-track>
    <protvista-track id="track-plddt" length="0" displaystart="1" displayend="0" shape="rectangle" label="pLDDT (optional)"    layout="compact"></protvista-track>

    <h4 style="margin:14px 0 6px">Scop3P phosphosites</h4>
    <protvista-track id="track-psite" length="0" displaystart="1" displayend="0" shape="triangle"  label="Phospho sites" layout="compact"></protvista-track>

    <h4 style="margin:14px 0 6px">Domain tracks</h4>
    <protvista-track id="track-pfam" length="0" displaystart="1" displayend="0" shape="rectangle" label="Pfam" layout="compact"></protvista-track>
    <protvista-track id="track-cath" length="0" displaystart="1" displayend="0" shape="rectangle" label="CATH" layout="compact"></protvista-track>

    <pre id="err" style="color:#b00;white-space:pre-wrap"></pre>
  </div>

  <div>
    <!-- Mol* -->
    <script src="https://www.ebi.ac.uk/pdbe/pdbe-molstar/build/pdbe-molstar.js"></script>
    <div id="molstar" style="height:100vh"></div>
  </div>
</div>

<script>
(async () => {
  try {
    const PACK = __JSON__;
    const $ = (q) => document.querySelector(q);
    const pv  = $("#pv-u");
    const sel = $("#structSel");
    const trackCons  = $("#track-cons");
    const trackDiso  = $("#track-diso");
    const trackPLDDT = $("#track-plddt");
    const trackPsite = $("#track-psite");
    const trackPfam  = $("#track-pfam");
    const trackCath  = $("#track-cath");

    const viewer = new PDBeMolstarPlugin();
    let up2pdb = PACK.mapping || {};
    let currentPdb = null;

    function populateStructs(){
      sel.innerHTML = "";
      if (PACK?.structures?.alphafold) {
        const o = document.createElement("option");
        o.value = "AF:" + PACK.structures.alphafold;
        o.textContent = "AlphaFold • " + PACK.structures.alphafold;
        sel.appendChild(o);
      }
      (PACK?.structures?.pdbs || []).forEach(p => {
        const o = document.createElement("option");
        o.value = "PDB:" + p;
        o.textContent = "PDB • " + p;
        sel.appendChild(o);
      });

      // ✅ robust preselect
      try {
        if (PACK.mapping && typeof PACK.mapping === "object") {
          const vals = Object.values(PACK.mapping);
          if (vals.length) {
            const firstHit = vals.find(v => Array.isArray(v) && v.length > 0);
            if (firstHit && firstHit[0] && firstHit[0].pdb) {
              const pid = String(firstHit[0].pdb).toUpperCase();
              const opt = [...sel.options].find(x => x.value === "PDB:" + pid);
              if (opt) sel.value = opt.value;
            }
          }
        }
      } catch(e) {
        console.warn("Preselect failed:", e);
      }
    }

    function scoreToHex(x){
      x = Math.max(0, Math.min(1, Number(x)||0));
      const r = Math.round(221 + (255-221)*x);
      const g = Math.round(221 + (0-221)*x);
      const b = Math.round(221 + (0-221)*x);
      return `#${r.toString(16).padStart(2,'0')}${g.toString(16).padStart(2,'0')}${b.toString(16).padStart(2,'0')}`;
    }

    async function uniLen(acc){
      try {
        const u = await (await fetch(`https://rest.uniprot.org/uniprotkb/${acc}.json`)).json();
        return u?.sequence?.length || u?.sequence?.value?.length || u?.sequence?.sequence?.length || 0;
      } catch(e) { return 0; }
    }

    async function renderScoreTrack(trackEl, label, arr, acc){
      const L = await uniLen(acc);
      trackEl.setAttribute("length", L);
      trackEl.setAttribute("displayend", L);
      const features = (arr||[]).map(d => ({
        accession: acc,
        type: label,
        color: scoreToHex(d.score ?? 0),
        locations: [{ fragments: [{ start: d.start, end: d.end }] }],
        tooltipContent: `${label}: ${d.score}`
      }));
      trackEl.data = { sequenceLength: L, features };
    }

    async function renderSiteTrack(trackEl, label, pos, acc){
      const L = await uniLen(acc);
      trackEl.setAttribute("length", L);
      trackEl.setAttribute("displayend", L);
      const features = (pos||[]).map(p => ({
        accession: acc,
        type: label,
        color: "#cc3355",
        locations: [{ fragments: [{ start: p, end: p }] }],
        tooltipContent: `Phospho @ ${p}`
      }));
      trackEl.data = { sequenceLength: L, features };
    }

    async function renderRangeTrack(trackEl, label, ranges, acc, color){
      const L = await uniLen(acc);
      trackEl.setAttribute("length", L);
      trackEl.setAttribute("displayend", L);
      const features = (ranges||[]).map(s => ({
        accession: acc,
        type: label,
        color: color,
        locations: [{ fragments: [{ start: s.unp_start, end: s.unp_end }] }],
        tooltipContent: `${(s.pfam_id || s.cath_id) ?? ''} ${s.unp_start}-${s.unp_end}`
      }));
      trackEl.data = { sequenceLength: L, features };
    }

    async function loadSelection(){
      const val = (sel.value || "");
      if (!val) return;
      if (val.startsWith("PDB:")) {
        const pdb = val.slice(4).toLowerCase();
        currentPdb = pdb;
        viewer.render(document.getElementById('molstar'), {
          moleculeId: pdb,
          hideControls: true,
          alphafoldView: false
        });
      } else if (val.startsWith("AF:")) {
        const afId = val.slice(3);
        currentPdb = null;
        viewer.render(document.getElementById('molstar'), {
          hideControls: true,
          alphafoldView: true,
          customData: { url: `https://alphafold.ebi.ac.uk/files/${afId}.cif`, format: "cif", binary: false }
        });
      }
    }

    // ✅ guard viewer.visual usage
    async function highlightUniProtRange(uStart, uEnd){
      if (!viewer || !viewer.visual) return;
      const loci = [];

      if (currentPdb) {
        if (up2pdb && typeof up2pdb === "object") {
          for (let u=uStart; u<=uEnd; u++) {
            const arr = up2pdb[u];
            if (Array.isArray(arr)) {
              for (const h of arr) {
                if (h && h.pdb === currentPdb && h.chain_label && h.label_seq_id != null) {
                  loci.push({ struct_asym_id: String(h.chain_label), seq_id: Number(h.label_seq_id) });
                }
              }
            }
          }
        }
      } else {
        for (let u=uStart; u<=uEnd; u++) loci.push({ struct_asym_id:"A", seq_id:u });
      }

      if (!loci.length) return;

      try {
        const selobj = { data: loci };
        viewer.visual.clearSelection?.();
        viewer.visual.select?.(selobj);
        viewer.visual.focus?.(selobj);
      } catch (e) {
        console.warn("Mol* highlight failed:", e);
      }
    }

    // Events
    pv.addEventListener('change', (e) => {
      const r = e.detail?.highlight?.[0];
      if (r?.start && r?.end) highlightUniProtRange(Number(r.start), Number(r.end));
    });
    pv.addEventListener('featureclick', (e) => {
      const f = e.detail?.feature;
      const loc = (f?.locations && f.locations[0]) || f?.location;
      const s = Number(loc?.start?.position || loc?.start), epos = Number(loc?.end?.position || loc?.end);
      if (s && epos) highlightUniProtRange(s, epos);
    });

    // ✅ ensure custom elements are ready
    await Promise.allSettled([
      customElements.whenDefined('protvista-uniprot'),
      customElements.whenDefined('protvista-track'),
    ]);

    // Initial render
    populateStructs();
    await renderScoreTrack(trackCons,  "Conservation",        PACK.tracks.conservation, PACK.uniprot);
    await renderScoreTrack(trackDiso,  "Disorder (DisoMine)", PACK.tracks.disorder,     PACK.uniprot);
    await renderScoreTrack(trackPLDDT, "pLDDT (optional)",    PACK.tracks.plddt,        PACK.uniprot);
    await renderSiteTrack (trackPsite, "Phospho",             PACK.scop3p.phospho_positions, PACK.uniprot);
    await renderRangeTrack(trackPfam,  "Pfam", PACK.pfam || [], PACK.uniprot, "#4f7cac");
    await renderRangeTrack(trackCath,  "CATH", PACK.cath || [], PACK.uniprot, "#7c4fa8");
    await loadSelection();

    const hb = document.createElement('div');
    hb.textContent = "UI loaded ✔";
    hb.style = "position:fixed;bottom:6px;left:8px;font:12px monospace;color:#777";
    document.body.appendChild(hb);
  } catch (e) {
    const errBox = document.getElementById('err');
    if (errBox) errBox.textContent = (e && (e.stack || e.message)) || String(e);
    console.error("Global error:", e);
  }
})();
</script>


</body>
</html>
"""
    html_filled = page.replace("__UNIPROT__", PACK["uniprot"]).replace("__JSON__", json.dumps(PACK))
    srcdoc = html.escape(html_filled, quote=True)
    iframe = f'<iframe srcdoc="{srcdoc}" style="width:100%;height:820px;border:1px solid #ddd;border-radius:8px"></iframe>'
    display(HTML(iframe))

@out.capture(clear_output=True)
def on_click(_):
    choice = sel.value
    # Build PACK for AF vs PDB/CHAIN (uses your existing parse function)
    if choice.startswith("AF:"):
        PACK = {
            "uniprot": UNI,
            "structures": {"alphafold": choice.split("AF:",1)[1], "pdbs": pdb_ids},
            "tracks": {"conservation": [], "disorder": [], "plddt": []},
            "scop3p": {"phospho_positions": fetch_scop3p_mods(UNI)[1]},
            "pfam": [], "cath": [], "mapping": {}
        }
    else:
        pdb_id, chain = choice.split("/")
        m = parse_updated_mmcif_for_chain(pdb_id, UNI, chain)  # your parser
        PACK = {
            "uniprot": UNI,
            "structures": {"alphafold": af_id, "pdbs": pdb_ids},
            "tracks": {"conservation": [], "disorder": [], "plddt": []},
            "scop3p": {"phospho_positions": fetch_scop3p_mods(UNI)[1]},
            "pfam": m["pfam_segments"], "cath": m["cath_segments"], "mapping": m["up2pdb"]
        }
    render_PACK(PACK)  # your Step-2 HTML renderer

btn.on_click(on_click) 
print("Select an entry from the dropdown and click 'Load selection'.")

UniProt O00571: 15 PDB IDs; dropdown options: 47


Dropdown(description='Structure', layout=Layout(width='460px'), options=('AF:AF-O00571-F1-model_v4', '2I4I/A',…

Button(description='Load selection', layout=Layout(width='200px'), style=ButtonStyle())

Output()

Select an entry from the dropdown and click 'Load selection'.


In [29]:
# --- UniProt → PDB/Chain viewer (local Mol*), with PFAM/CATH labels, colored phospho, grey non-mapped chains,
#     and a zoomable 1D sequence strip with residue letters (clickable) ---
import os, json, requests
from IPython.display import HTML, display

# -------- server-side helpers --------
def fetch_uniprot_entry(acc):
    r = requests.get(f"https://rest.uniprot.org/uniprotkb/{acc}.json",
                     headers={"Accept":"application/json","User-Agent":"scop3p-local/1.0"}, timeout=30)
    r.raise_for_status()
    return r.json()

def uniprot_pdb_ids(acc):
    j = fetch_uniprot_entry(acc)
    x = j.get("uniProtKBCrossReferences", [])
    return sorted({i["id"] for i in x if i.get("database") == "PDB"})

def fetch_scop3p_phospho(acc):
    try:
        r = requests.get(f"https://iomics.ugent.be/scop3p/api/modifications?accession={acc}",
                         headers={"Accept":"application/json"}, timeout=30)
        r.raise_for_status()
        mods = (r.json() or {}).get("modifications", [])
        return sorted({m["position"] for m in mods
                       if isinstance(m, dict) and "phospho" in (m.get("name","").lower())
                       and isinstance(m.get("position"), int)})
    except Exception:
        return []

def pfam_name(pfam_id):
    try:
        r = requests.get(f"https://www.ebi.ac.uk/interpro/api/entry/pfam/{pfam_id}",
                         headers={"Accept":"application/json"}, timeout=20)
        if r.ok:
            js = r.json()
            nm = (js.get("metadata") or {}).get("name") or js.get("name")
            if isinstance(nm, str) and nm.strip(): return nm.strip()
    except Exception:
        pass
    try:
        r = requests.get(f"https://pfam.xfam.org/family/{pfam_id}?output=json",
                         headers={"Accept":"application/json","User-Agent":"scop3p-local/1.0"}, timeout=20)
        if r.ok:
            js = r.json()
            desc = (js.get("entry", {}).get("description") or "").strip()
            if desc: return desc
    except Exception:
        pass
    return str(pfam_id)

def cath_name(cath_id):
    try:
        r = requests.get(f"https://www.cathdb.info/api/rest/superfamily/{cath_id}",
                         headers={"Accept":"application/json"}, timeout=20)
        if r.ok:
            js = r.json()
            nm = js.get("name") or js.get("description")
            if isinstance(nm, str) and nm.strip(): return nm.strip()
    except Exception:
        pass
    return str(cath_id)

def read_cat(block, cat):
    try:
        import pandas as pd
        tbl = block.get_mmcif_category(cat)
        return pd.DataFrame(tbl) if tbl is not None else pd.DataFrame()
    except Exception:
        return None

def parse_updated_mmcif_all_for_uniprot(pdb_id, uniprot_acc, cif_path):
    """
    Return:
      chains: [{"id": chain_label, "length": max_label_seq_id}, ...]
      mapping: { unp_pos: [ { 'pdb', 'chain_label', 'label_seq_id' } ... ] }
      pfam_segments: [{unp_start, unp_end, pfam_id, pfam_name}]
      cath_segments: [{unp_start, unp_end, cath_id, cath_name}]
    """
    from gemmi import cif as gemmi_cif
    import pandas as pd

    doc  = gemmi_cif.read_file(cif_path)
    block = doc.sole_block()
    poly = read_cat(block, "_pdbx_poly_seq_scheme")
    atom = read_cat(block, "_atom_site")

    chains = []
    if poly is not None and not poly.empty and "asym_id" in poly.columns and "seq_id" in poly.columns:
        poly2 = poly[["asym_id","seq_id"]].copy()
        poly2["seq_id"] = pd.to_numeric(poly2["seq_id"], errors="coerce")
        poly2 = poly2.dropna()
        lens = poly2.groupby("asym_id")["seq_id"].max().astype(int).to_dict()
        chains = [{"id": str(a), "length": int(n)} for a, n in sorted(lens.items())]
    elif atom is not None and not atom.empty and {"label_asym_id","label_seq_id"} <= set(atom.columns):
        atom2 = atom[["label_asym_id","label_seq_id"]].dropna()
        atom2["label_seq_id"] = pd.to_numeric(atom2["label_seq_id"], errors="coerce")
        atom2 = atom2.dropna()
        lens = atom2.groupby("label_asym_id")["label_seq_id"].max().astype(int).to_dict()
        chains = [{"id": str(a), "length": int(n)} for a, n in sorted(lens.items())]

    sifts = read_cat(block, "_pdbx_sifts_xref_db")
    up2pdb = {}
    pfam_segments, cath_segments = [], []

    if sifts is not None and not sifts.empty:
        sifts = sifts.rename(columns=str)
        for c in ["asym_id","seq_id","unp_acc","unp_num","xref_db_name","xref_db_acc"]:
            if c not in sifts.columns: sifts[c] = ""
        # mapping rows
        unp = sifts[(sifts["unp_acc"] == uniprot_acc) & (sifts["seq_id"] != "") & (sifts["unp_num"] != "")]
        if not unp.empty:
            unp["seq_id"] = pd.to_numeric(unp["seq_id"], errors="coerce").astype("Int64")
            unp["unp_num"] = pd.to_numeric(unp["unp_num"], errors="coerce").astype("Int64")
            unp = unp.dropna(subset=["seq_id","unp_num"]).astype({"seq_id": int, "unp_num": int})
            for r in unp.itertuples(index=False):
                up2pdb.setdefault(int(r.unp_num), []).append({
                    "pdb": pdb_id.lower(),
                    "chain_label": str(r.asym_id),
                    "label_seq_id": int(r.seq_id)
                })

        # domains with names
        def collapse_with_names(df, dbname, id_field, name_func):
            sub = df[(df["xref_db_name"].str.upper()==dbname) & (df["unp_acc"]==uniprot_acc)][["xref_db_acc","unp_num"]].dropna()
            if sub.empty: return []
            sub["unp_num"] = sub["unp_num"].astype(int)
            segs = []
            for dom, grp in sub.sort_values(["xref_db_acc","unp_num"]).groupby("xref_db_acc"):
                dom = str(dom)
                pos = grp["unp_num"].tolist()
                s = prev = pos[0]
                for p in pos[1:]:
                    if p != prev+1:
                        segs.append({"unp_start": int(s), "unp_end": int(prev), id_field: dom})
                        s = p
                    prev = p
                segs.append({"unp_start": int(s), "unp_end": int(prev), id_field: dom})
            name_key = id_field.replace("_id","_name")
            for seg in segs:
                try:
                    nm = name_func(seg[id_field])
                except Exception:
                    nm = None
                seg[name_key] = str(nm) if isinstance(nm, (str,int,float)) else str(seg[id_field])
            return segs

        pfam_segments = collapse_with_names(sifts, "PFAM", "pfam_id", pfam_name)
        cath_segments = collapse_with_names(sifts, "CATH", "cath_id", cath_name)

    return chains, up2pdb, pfam_segments, cath_segments

def write_local_dropdown_viewer_v3(uniprot="O00571", limit_pdb=None, folder="pv_local_dropdown_v3"):
    os.makedirs(folder, exist_ok=True)

    # UniProt basics
    u = fetch_uniprot_entry(uniprot)
    L = u.get("sequence",{}).get("length") or (u.get("sequence",{}).get("value") and len(u["sequence"]["value"])) or 1000
    seq = u.get("sequence",{}).get("value") or u.get("sequence",{}).get("sequence") or ""
    phospho = fetch_scop3p_phospho(uniprot)

    pdb_ids = uniprot_pdb_ids(uniprot)
    if limit_pdb: pdb_ids = pdb_ids[:int(limit_pdb)]

    structures, mapping_all = {}, {}
    pfam_union, cath_union = [], []

    # Download + parse each PDB updated mmCIF
    for pid in pdb_ids:
        low = pid.lower()
        cif_path = os.path.join(folder, f"{low}_updated.cif")
        if not os.path.exists(cif_path):
            url = f"https://www.ebi.ac.uk/pdbe/entry-files/{low}_updated.cif"
            r = requests.get(url, timeout=60)
            if not r.ok or not r.content:
                continue
            with open(cif_path, "wb") as f:
                f.write(r.content)
        try:
            chains, up2pdb, pfam, cath = parse_updated_mmcif_all_for_uniprot(pid, uniprot, cif_path)
        except Exception:
            chains, up2pdb, pfam, cath = [], {}, [], []
        structures[pid] = {"file": f"{low}_updated.cif", "chains": chains}
        mapping_all[pid] = up2pdb
        pfam_union += pfam
        cath_union += cath

    # dedup ranges (hashable primitives only)
    def dedup(segs, idk):
        seen, out = set(), []
        namek = idk.replace("_id","_name")
        for s in segs:
            start = int(s.get("unp_start", 0) or 0)
            end   = int(s.get("unp_end",   0) or 0)
            dom   = str(s.get(idk, ""))
            name  = str(s.get(namek, dom))
            key = (start, end, dom, name)
            if key in seen: 
                continue
            seen.add(key)
            out.append({"unp_start": start, "unp_end": end, idk: dom, namek: name})
        return out

    PACK = {
        "uniprot": uniprot,
        "length": int(L),
        "sequence": seq,
        "phospho": phospho,
        "structures": structures,
        "mapping": mapping_all,
        "pfam": dedup(pfam_union, "pfam_id"),
        "cath": dedup(cath_union, "cath_id")
    }

    # Local Mol* assets
    assets = {
        "pdbe-molstar-plugin.js": "https://cdn.jsdelivr.net/npm/pdbe-molstar@latest/build/pdbe-molstar-plugin.js",
        "pdbe-molstar.css":       "https://cdn.jsdelivr.net/npm/pdbe-molstar@latest/build/pdbe-molstar.css",
    }
    for fname, url in assets.items():
        path = os.path.join(folder, fname)
        if not os.path.exists(path):
            r = requests.get(url, timeout=60); r.raise_for_status()
            with open(path, "wb") as f: f.write(r.content)

    # HTML+JS
    page = f"""<!doctype html>
<html>
<head>
<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<title>UniProt → PDB/Chain viewer (local)</title>
<link rel="stylesheet" href="pdbe-molstar.css"/>
<style>
  html, body {{ height:100%; margin:0; font-family: ui-sans-serif, system-ui; background:#fff; }}
  #wrap {{ display:flex; height:100vh; width:100vw; overflow:hidden; }}
  #left {{ width: 620px; flex:0 0 620px; padding:12px; border-right:1px solid #ddd; background:#fff; overflow:auto; }}
  #right {{ flex:1 1 auto; position:relative; background:#f7f7f8; overflow:hidden; }}
  .row {{ display:flex; gap:8px; align-items:center; margin:0 0 12px; flex-wrap: wrap; }}
  select, input[type="range"] {{ padding:6px 8px; border:1px solid #ccc; border-radius:8px; }}
  label {{ font-size:12px; color:#444; }}
  .h3 {{ margin:8px 0 6px; font-size:16px; font-weight:600; }}
  .track {{ margin:8px 0 16px; }}
  .svgbox {{ background:#fafafa; border:1px solid #eee; border-radius:6px; }}
  .legend {{ font:12px/1.4 monospace; color:#555; margin-top:4px; }}
  .badge {{ display:inline-block; padding:2px 6px; font-size:11px; border-radius:10px; color:#fff; }}
  #molstar {{ position:absolute; inset: 44px 0 0 0; }}
  #topbar {{ position:absolute; left:0; top:0; right:0; height:44px; background:#fff; border-bottom:1px solid #e9e9e9;
             display:flex; align-items:center; gap:10px; padding:0 10px; z-index:3; }}
  .muted {{ color:#999; }}
  .tiny {{ font:11px ui-sans-serif, system-ui; color:#666; }}
</style>
</head>
<body>
<div id="wrap">
  <div id="left">
    <div class="row">
      <label>PDB:</label>
      <select id="pdbSel"></select>
      <label>Chain:</label>
      <select id="chainSel"></select>
      <label style="margin-left:8px;display:flex;align-items:center;gap:6px">
        <input type="checkbox" id="allChains" checked/> All chains
      </label>
      <label style="margin-left:8px;display:flex;align-items:center;gap:6px">
        <input type="checkbox" id="greyUnmapped" checked/> Grey non-mapped chains
      </label>
      <button id="showPhospho" style="margin-left:auto;padding:6px 10px;border:1px solid #ccc;border-radius:8px;background:#fff">Show phospho</button>
    </div>

    <div class="h3">1D (UniProt {PACK["uniprot"]})</div>

    <!-- Zoomable sequence letters -->
    <div class="row tiny">
      <span>Sequence letters:</span>
      <span style="margin-left:8px">Start</span> <input id="seqStart" type="range" min="1" max="{PACK['length']}" value="1" style="width:180px"/>
      <span>Window</span> <input id="seqWindow" type="range" min="10" max="200" value="60" style="width:180px"/>
      <span id="seqInfo"></span>
    </div>
    <div id="seqLetters" class="track"></div>

    <!-- Domain / site tracks -->
    <div id="tracks"></div>

    <div class="legend">
      <div><span class="badge" style="background:#4f7cac">PFAM</span> domain (click block) — shows ID + name</div>
      <div><span class="badge" style="background:#7c4fa8">CATH</span> domain (click block) — shows ID + name</div>
      <div><span class="badge" style="background:#ffb300">P</span> phospho site (click site) — colored amber</div>
    </div>

    <div id="status" style="margin-top:10px;color:#777;font:12px monospace">loading…</div>
    <pre id="err" style="color:#b00;white-space:pre-wrap"></pre>
  </div>

  <div id="right">
    <div id="topbar">
      <span style="font:12px/1 ui-sans-serif, system-ui;color:#444">Mol* (PDBe plugin)</span>
    </div>
    <script src="pdbe-molstar-plugin.js"></script>
    <div id="molstar"></div>
  </div>
</div>

<script>
  const PACK = {json.dumps(PACK)};
  const COLORS = {{
    phospho: 0xffd54f,   // amber
    click:   0x1976d2,   // blue
    grey:    0xbdbdbd    // grey for non-mapped chains
  }};
  let viewer = null;
  let currentPdb = null;
  let currentChain = null;

  // ---------- DOM + SVG helpers ----------
  const $ = (q) => document.querySelector(q);
  function mkSVG(w,h){{const s=document.createElementNS("http://www.w3.org/2000/svg","svg");s.setAttribute("width",w);s.setAttribute("height",h);s.classList.add("svgbox");return s;}}
  function line(svg,x1,y1,x2,y2,stroke,w){{const l=document.createElementNS(svg.namespaceURI,"line");l.setAttribute("x1",x1);l.setAttribute("y1",y1);l.setAttribute("x2",x2);l.setAttribute("y2",y2);l.setAttribute("stroke",stroke);l.setAttribute("stroke-width",w);svg.appendChild(l);return l;}}
  function rect(svg,x,y,w,h,fill){{const r=document.createElementNS(svg.namespaceURI,"rect");r.setAttribute("x",x);r.setAttribute("y",y);r.setAttribute("width",w);r.setAttribute("height",h);r.setAttribute("fill",fill);r.setAttribute("rx",3);r.setAttribute("ry",3);svg.appendChild(r);return r;}}
  function text(svg,x,y,str,fill,size,anchor){{const t=document.createElementNS(svg.namespaceURI,"text");t.setAttribute("x",x);t.setAttribute("y",y);t.setAttribute("fill",fill||"#222");t.setAttribute("font-size",size||11);t.setAttribute("font-family","ui-sans-serif, system-ui");if(anchor) t.setAttribute("text-anchor",anchor);t.textContent=str;svg.appendChild(t);return t;}}

  // ---------- Mol* ----------
  function loadMolstar(pdbId, cifFile){{
    try {{
      if (!window.PDBeMolstarPlugin) {{ setTimeout(()=>loadMolstar(pdbId,cifFile),50); return; }}
      if (!viewer) viewer = new PDBeMolstarPlugin();
      viewer.render($("#molstar"), {{ hideControls:false, alphafoldView:false, customData:{{url:cifFile, format:"cif", binary:false}} }});
      $("#status").textContent = "Mol* loaded ✔ " + pdbId + (currentChain ? ("/"+currentChain) : "");
      setTimeout(() => {{ recolorGreyNonMapped(); applyPhosphoBaseHighlight(); }}, 400);
    }} catch (e) {{
      $("#status").textContent = "Mol* error"; $("#err").textContent = (e && (e.stack || e.message)) || String(e); console.error(e);
    }}
  }}
  function getMapForCurrentPdb(){{ return PACK.mapping[currentPdb] || PACK.mapping[currentPdb?.toUpperCase?.()] || {{}}; }}

  function lociFromUniProtRange(u1,u2, allChains){{
    const map = getMapForCurrentPdb(); const loci=[];
    for (let u=u1; u<=u2; u++){{ const arr=map[u]; if(!Array.isArray(arr)) continue;
      for(const h of arr){{ if(!h||!h.chain_label||h.label_seq_id==null) continue;
        if(!allChains && String(h.chain_label)!==String(currentChain)) continue;
        loci.push({{ struct_asym_id:String(h.chain_label), seq_id:Number(h.label_seq_id) }});
      }}
    }} return loci;
  }}

  // Base amber highlight for Scop3P across all mapped chains
  function applyPhosphoBaseHighlight(){{
    if(!viewer||!viewer.visual||!currentPdb) return;
    const loci=[]; (PACK.phospho||[]).forEach(up=>loci.push(...lociFromUniProtRange(up,up,true)));
    if(!loci.length) return;
    try {{
      // Keep grey colors; add amber on top
      viewer.visual.select({{ data: loci, color: COLORS.phospho, keepColors: true, keepRepresentations: true }});
    }} catch(e) {{ console.warn("Phospho highlight failed", e); }}
  }}

  // Grey out chains that don't map any UniProt residue
  function recolorGreyNonMapped(){{
    if(!$("#greyUnmapped").checked) return;
    if(!viewer||!viewer.visual||!currentPdb) return;
    const s = PACK.structures[currentPdb] || {{}};
    const cs = (s.chains||[]).map(c => String(c.id));
    const map = getMapForCurrentPdb();
    const mapped = new Set();
    Object.values(map).forEach(arr => {{ if(Array.isArray(arr)) arr.forEach(h => h?.chain_label && mapped.add(String(h.chain_label))); }});
    const unmapped = cs.filter(id => !mapped.has(id));
    const loci=[];
    // color entire chain range 1..length
    (s.chains||[]).forEach(c => {{
      const id = String(c.id), L = Number(c.length||0);
      if (!unmapped.includes(id) || !L || L<1) return;
      for(let i=1;i<=L;i++) loci.push({{ struct_asym_id: id, seq_id: i }});
    }});
    if(!loci.length) return;
    try {{
      viewer.visual.select({{ data: loci, color: COLORS.grey, keepColors: true, keepRepresentations: true }});
    }} catch(e) {{ console.warn("Grey unmapped failed", e); }}
  }}

  function clearSelectionsPreservingBase(){{
    if(!viewer||!viewer.visual) return;
    try {{
      viewer.visual.clearSelection();
      // reapply base layers in order: grey → phospho
      recolorGreyNonMapped();
      applyPhosphoBaseHighlight();
    }} catch(e){{ console.warn("clearSelectionsPreservingBase failed", e); }}
  }}

  function selectAndFocusLoci(loci){{
    if(!viewer||!viewer.visual||!loci.length) return;
    try {{
      // keep base; overlay click color
      viewer.visual.select({{ data: loci, color: COLORS.click, keepColors: true, keepRepresentations: true }});
      viewer.visual.focus({{ data: loci }});
    }} catch(e){{ console.warn("Mol* selection failed", e); }}
  }}

  // ---------- Tracks ----------
  function drawDomainTrack(label, color, segs, idk, namek, L){{
    if(!Array.isArray(segs) || !segs.length) return;
    const box=$("#tracks"); const width=Math.max(520, box.getBoundingClientRect().width-20); const height=34;
    const svg=mkSVG(width,height); const y=8, h=height-16;
    segs.forEach(s=>{{
      const s1=Math.max(1,Number(s.unp_start||0)); const e1=Math.max(s1,Number(s.unp_end||0));
      const x=10+(s1-1)/(L-1)*(width-20); const w=Math.max(2,(e1-s1+1)/L*(width-20));
      const rr=rect(svg,x,y,w,h,color); rr.style.cursor="pointer";
      const id=(s[idk]||""); const nm=(s[namek]||"").trim();
      rr.title = (id?id:"") + (nm?(" • "+nm):"");
      // centered label, clipped
      const labelTxt = (id + (nm?(" • "+nm):"")).slice(0, 40);
      const t = text(svg, x+w/2, y+h/2+4, labelTxt, "#fff", 11, "middle"); t.style.pointerEvents="none";
      rr.addEventListener("click", ()=>{{ const loci=lociFromUniProtRange(s1,e1,$("#allChains").checked); clearSelectionsPreservingBase(); selectAndFocusLoci(loci); }});
    }});
    const wrap=document.createElement("div"); wrap.className="track";
    const lab=document.createElement("div"); lab.textContent = label + " (click block)"; lab.style.font="12px ui-sans-serif, system-ui"; lab.style.margin="0 0 4px";
    wrap.appendChild(lab); wrap.appendChild(svg); $("#tracks").appendChild(wrap);
  }}

  function drawPhosphoTrack(L){{
    if(!Array.isArray(PACK.phospho)||!PACK.phospho.length) return;
    const box=$("#tracks"); const width=Math.max(520, box.getBoundingClientRect().width-20); const height=30;
    const svg=mkSVG(width,height);
    PACK.phospho.forEach(up=>{{ const x=10+(up-1)/(L-1)*(width-20);
      const l=line(svg,x,6,x,height-6,"#ffb300",2); l.style.cursor="pointer"; l.title="Phospho @ "+up;
      l.addEventListener("click", ()=>{{ const loci=lociFromUniProtRange(up,up,$("#allChains").checked); clearSelectionsPreservingBase(); selectAndFocusLoci(loci); }});
    }});
    const wrap=document.createElement("div"); wrap.className="track";
    const lab=document.createElement("div"); lab.textContent="Phospho (click site)"; lab.style.font="12px ui-sans-serif, system-ui"; lab.style.margin="0 0 4px";
    wrap.appendChild(lab); wrap.appendChild(svg); $("#tracks").appendChild(wrap);
  }}

  function drawTracks(){{
    const L = Math.max(1, Number(PACK.length||0));
    $("#tracks").innerHTML = "";
    // UniProt axis with ticks
    (function(){{
      const width=Math.max(520, $("#tracks").getBoundingClientRect().width-20); const height=30; const svg=mkSVG(width,height); const y=height*0.5;
      line(svg,10,y,width-10,y,"#444",2);
      for(let i=0;i<=5;i++){{const p=i/5; const x=10+p*(width-20); line(svg,x,y-6,x,y+6,"#aaa",1); text(svg,x-8,height-4,String(Math.round(L*p)),"#666",10);}}
      const wrap=document.createElement("div"); wrap.className="track";
      const lab=document.createElement("div"); lab.textContent="UniProt axis"; lab.style.font="12px ui-sans-serif, system-ui"; lab.style.margin="0 0 4px";
      wrap.appendChild(lab); wrap.appendChild(svg); $("#tracks").appendChild(wrap);
    }})();
    drawDomainTrack("Pfam", "#4f7cac", PACK.pfam, "pfam_id", "pfam_name", L);
    drawDomainTrack("CATH", "#7c4fa8", PACK.cath, "cath_id", "cath_name", L);
    drawPhosphoTrack(L);
  }}

  // ---------- Sequence letters with zoom ----------
  function drawSequenceLetters(){{
    const seq = String(PACK.sequence||""); const L = seq.length || Number(PACK.length||0);
    const start = Math.max(1, Number($("#seqStart").value||1));
    const win   = Math.max(10, Number($("#seqWindow").value||60));
    const end   = Math.min(L, start + win - 1);
    $("#seqInfo").textContent = " " + start + "-" + end + " / " + L;

    const box = $("#seqLetters"); const width = Math.max(520, $("#left").getBoundingClientRect().width - 40); const height = 40;
    box.innerHTML = "";
    const svg = mkSVG(width, height);

    const content = seq ? seq.slice(start-1, end) : "";
    const n = end - start + 1;
    const pad = 12, usable = width - 2*pad;
    const cell = usable / Math.max(1, n);

    // background strip
    rect(svg, pad, 8, usable, height-16, "#fdfdfd");
    line(svg, pad, height-12, pad+usable, height-12, "#ddd", 1);

    for (let i=0;i<n;i++) {{
      const up = start + i;
      const aa = content ? content[i] : "";
      const x  = pad + i*cell + cell*0.5;
      const g  = document.createElementNS(svg.namespaceURI,"g");
      const tick = line(svg, x, 12, x, height-12, "#eee", 1); tick.setAttribute("opacity","0.8");
      const t = text(svg, x, 26, aa||"", "#222", 12, "middle"); t.style.cursor="pointer";
      t.addEventListener("click", ()=>{{ const loci=lociFromUniProtRange(up, up, $("#allChains").checked); clearSelectionsPreservingBase(); selectAndFocusLoci(loci); }});
      svg.appendChild(g);
    }}
    // ticks every 10
    for (let up = ((Math.ceil(start/10))*10); up <= end; up += 10) {{
      const i = up - start; if (i < 0 || i >= n) continue;
      const x = pad + i*cell + cell*0.5;
      text(svg, x, height-4, String(up), "#666", 10, "middle");
    }}

    box.appendChild(svg);
  }}

  // ---------- Dropdowns + boot ----------
  function populatePdbs(){{
    const sel=$("#pdbSel"); sel.innerHTML=""; const ids=Object.keys(PACK.structures).sort();
    ids.forEach(pid=>{{ const o=document.createElement("option"); o.value=pid; o.textContent=pid; sel.appendChild(o); }});
    if(ids.length) sel.value=ids[0];
  }}
  function populateChains(){{
    const pdb=$("#pdbSel").value; const cs=((PACK.structures[pdb]||{{}}).chains)||[];
    const map=PACK.mapping[pdb] || PACK.mapping[pdb?.toUpperCase?.()] || {{}}; const mapped=new Set();
    Object.values(map).forEach(arr=>{{ if(Array.isArray(arr)) arr.forEach(h=>h?.chain_label && mapped.add(String(h.chain_label))); }});
    const sel=$("#chainSel"); sel.innerHTML="";
    cs.forEach(c=>{{ const id=String(c.id); const lab = id+" ("+(c.length||"")+")"+(mapped.has(id)?"":" • no map");
      const o=document.createElement("option"); o.value=id; o.textContent=lab; if(!mapped.has(id)) o.className="muted"; sel.appendChild(o);
    }});
    if(cs.length) sel.value=String(cs[0].id);
  }}
  function onSelectionChanged(){{
    currentPdb=$("#pdbSel").value; currentChain=$("#chainSel").value;
    const s=PACK.structures[currentPdb] || {{}}; const cifFile=s.file || "";
    if(cifFile) loadMolstar(currentPdb, cifFile);
  }}

  window.addEventListener("DOMContentLoaded", () => {{
    try {{
      populatePdbs(); populateChains(); onSelectionChanged();
      drawTracks(); drawSequenceLetters();
      $("#pdbSel").addEventListener("change", () => {{ populateChains(); onSelectionChanged(); }});
      $("#chainSel").addEventListener("change", () => {{ onSelectionChanged(); }});
      $("#showPhospho").addEventListener("click", () => {{ clearSelectionsPreservingBase(); applyPhosphoBaseHighlight(); }});
      $("#allChains").addEventListener("change", () => {{ /* nothing to do until user clicks; base stays */ }});
      $("#greyUnmapped").addEventListener("change", () => {{ clearSelectionsPreservingBase(); }});
      $("#seqStart").addEventListener("input", drawSequenceLetters);
      $("#seqWindow").addEventListener("input", drawSequenceLetters);
      $("#status").textContent = "Ready";
    }} catch(e) {{
      $("#status").textContent="Init error"; $("#err").textContent=(e && (e.stack||e.message)) || String(e);
    }}
  }});
</script>
</body>
</html>"""

    with open(os.path.join(folder, "index.html"), "w", encoding="utf-8") as f:
        f.write(page)

    display(HTML(
        f'✅ Wrote <code>{folder}/index.html</code>. '
        f'Open <a href="http://localhost:8000/{folder}/index.html" target="_blank">'
        f'http://localhost:8000/{folder}/index.html</a>  (serve via: <code>python -m http.server 8000</code>)'
    ))

# Build & open
write_local_dropdown_viewer_v3("O00571", limit_pdb=None)


/tmp/ipykernel_16922/2758012890.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unp["seq_id"] = pd.to_numeric(unp["seq_id"], errors="coerce").astype("Int64")
/tmp/ipykernel_16922/2758012890.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unp["unp_num"] = pd.to_numeric(unp["unp_num"], errors="coerce").astype("Int64")
/tmp/ipykernel_16922/2758012890.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in